# About

This notebook is created to prepare data for classification if truck moving or not. It is required for clustering to ensure stability. We are 100% sure that moving vehicle will not collect the waste

In [3]:
# Imports
import os
import numpy as np
import pandas as pd
import logging
import plotly.express as px
import geopandas as gpd
import plotly.express as px
from multiprocessing import Pool
from GPSOdyssey import Polaris, Kepler, Void, Vega
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Data

In [20]:
S_MAP_MATCHING_REPORT = '/Data/Intermediate/MapMatchingReports/MapMatching_october2020.csv'

map_matching_report = pd.read_csv(S_MAP_MATCHING_REPORT, parse_dates=[
                                  'waste_collection_date']).sort_values(by=['waste_collection_date', 'truck_id'])

fnames = map_matching_report['csv_file'][:15]

## GPS records

In [29]:
%%time
S_CLUSTERED_GPS = '/Data/Source/OctoberGPS/'


vega = Vega(engine='pandas')
gps = vega.read_from_dir(directory=S_CLUSTERED_GPS,
                             file_extensions='.csv',
                             args={'parse_dates': ['time']},
                             concatenate=True, amt_in_parallel=6, filenames=fnames)

CPU times: user 81.9 ms, sys: 70.1 ms, total: 152 ms
Wall time: 178 ms


In [30]:
Polaris(gps) \
    .select_columns(['truck_id', 'lon', 'lat', 'lon_match', 'lat_match', 'time']) \
    .construct_datetime(datetime='time', time_col='datetime') \
    .add_date_col('datetime', 'date') \
    .add_time_col('datetime', 'time') \
    .add_unixtime

,truck_id,lon,lat,time,unixtime,index,alternatives_count,waypoint_index,matchings_index,name,...,matching_confidence,distance_traveled,duration,osm_nodes,osm_speeds,osm_durations,osm_distances,n_osm_points,speed,outlier
0,XE 5559L,103.753584,1.309354,2020-10-01 13:22:55,1601558575,0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,XE 5559L,103.753519,1.309244,2020-10-01 13:23:01,1601558581,1,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,XE 5559L,103.753460,1.309141,2020-10-01 13:23:06,1601558586,2,0.0,1.0,0.0,NaN,...,0.842883,26.1,6.2,"[5256785405, 4602541760]",[4.2],[6.2],[26.071836],2.0,4.209677,False
3,XE 5559L,103.753426,1.309110,2020-10-01 13:23:11,1601558591,3,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,XE 5559L,103.753407,1.309079,2020-10-01 13:23:16,1601558596,4,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71953,XE 5639P,103.648586,1.330315,2020-10-01 14:29:05,1601562545,6670,NaN,NaN,53.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
71954,XE 5639P,103.648586,1.330320,2020-10-01 14:29:10,1601562550,6671,NaN,NaN,53.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
71955,XE 5639P,103.648586,1.330321,2020-10-01 14:29:15,1601562555,6672,NaN,NaN,53.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
71956,XE 5639P,103.648583,1.330319,2020-10-01 14:29:21,1601562561,6673,NaN,NaN,53.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


# GPS visualization

In [28]:
# gps = gps[['truck_id', 'lon_match', 'lat_match', 'datetime', 'time', 'unixtime']]

# kepler = Kepler({'gps': gps},
#                 # config_path=MAP_CONFIG,
#                 height=800)
               
# kepler.render_kepler_map()
# kepler.get_rendered_map()

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'gps':        truck_id   lon_match  lat_match                 time    unixtime
0      XE 5559L …

# GPS Record slices